In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt   
from financepy.finutils import *
from financepy.market.curves import *
from financepy.products.equity import *

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

import warnings
warnings.filterwarnings('ignore')

### Setting up

In [ ]:
valueDate = FinDate(1, 9, 2020)
T = 1
sq_T = np.sqrt(T)
expiryDate = valueDate.addYears(T)

stockPrice = 10
dividendYield = 0.00
interestRate = 0.00
discountCurve = FinDiscountCurveFlat(valueDate, interestRate, FinFrequencyTypes.CONTINUOUS)

volatility = 0.20
model = FinEquityModelBlackScholes(volatility)

v_sq_T = volatility*volatility*T

### close form solution for E[S^2]
closed_form = stockPrice*stockPrice*np.exp(v_sq_T)
print(closed_form)

### How to choose the upper and lower bounds

In [ ]:
num_std = 5
upper_bound = stockPrice * np.exp(-0.5*v_sq_T + volatility*sq_T*num_std)
lower_bound = stockPrice * np.exp(-0.5*v_sq_T + volatility*sq_T*-num_std)

print(upper_bound)
print(lower_bound)

In [ ]:
def BL(stockPrice, max_k, num_ks):
    
    ks = np.linspace(1, max_k, num_ks)
    
    x = stockPrice
    greater_than_x = ks > x

    put_ks = ks[~greater_than_x]
    call_ks = ks[greater_than_x]    
    diff = call_ks[0] - put_ks[-1]
    
    dk_put = np.diff(put_ks)
    dk_put = np.append(dk_put, diff)  
    dk_call = np.diff(call_ks)

    # Create the objects for all put and call options
    putOptions = [FinEquityVanillaOption(expiryDate, k, FinOptionTypes.EUROPEAN_PUT) for k in put_ks]
    callOptions = [FinEquityVanillaOption(expiryDate, k, FinOptionTypes.EUROPEAN_CALL) for k in call_ks[:-1]]

    # Compute all the put and call option prices
    put_prices = [putOption.value(valueDate, stockPrice, discountCurve, dividendYield, model) for putOption in putOptions]
    call_prices = [callOption.value(valueDate, stockPrice, discountCurve, dividendYield, model) for callOption in callOptions]

    # Using the Breeden-Liztenberger Formula
    put_component = [put_prices[i] * dk for i, dk in enumerate(dk_put)]
    call_component = [call_prices[i] * dk for i, dk in enumerate(dk_call)]
    final_value = stockPrice*stockPrice + 2 * sum(put_component) + 2 * sum(call_component)
    
    return final_value

In [ ]:
@interact
def plot_BL_changing_max_k(num_k = (20, 100, 1)):    
    max_ks = np.linspace(15, 30, 10)    
    values = [BL(stockPrice, max_k, num_k) for max_k in max_ks]
    
    plt.plot(max_ks, values)   
    
    plt.axhline(closed_form, color='red')
    plt.ylim((closed_form*0.999, closed_form*1.001))
    plt.title("num_k = " + str(num_k)); plt.xlabel("max_k"); plt.ylabel("BL approximation"); plt.grid()

In [ ]:
@interact
def plot_BL_changing_num_k(max_k = (15, 50, 5)):    
    num_ks = np.linspace(20, 100, 10)    
    values = [BL(stockPrice, max_k, num_k) for num_k in num_ks]    
    plt.plot(num_ks, values)       
    plt.axhline(closed_form, color='red')
    plt.ylim((closed_form*0.999, closed_form*1.001))
    plt.title("max_k = " + str(max_k)); plt.xlabel("num_k"); plt.ylabel("BL approximation"); plt.grid()